# Bedrock AgentCore Gateway를 사용하여 OpenAPI 사양에서 MCP 구축하기

이 워크샵은 Amazon Bedrock AgentCore Gateway를 사용하여 OpenAPI 사양에서 Model Context Protocol (MCP) 서버를 자동으로 생성하는 방법을 보여주며, 

외부 API와 AI 에이전트의 원활한 통합을 가능하게 합니다.

## 개요

이 실습에서는 다음을 수행합니다:
- 인증이 포함된 Bedrock AgentCore Gateway 생성
- 외부 API 사양을 사용하여 OpenAPI 기반 게이트웨이 대상 구성
- 외부 API 액세스를 위한 보안 자격 증명 관리 설정
- Strands Agents와 함께 게이트웨이 배포 및 테스트
- OpenAPI 스키마에서 자동 MCP 도구 생성 탐색

## 사전 요구사항

이 실습을 시작하기 전에 다음이 준비되어 있는지 확인하세요:
- AWS 자격 증명 구성 (IAM 역할 또는 환경 변수)
- 필요한 Python 패키지 설치
- AWS 리전에 따른 Nova Pro 모델 ID
- 테스트용 외부 API 키 (예: Exa API 키)

IAM 역할이 가정된 환경에서 실행하지 않는 경우, AWS 자격 증명을 환경 변수로 설정하세요:

In [ ]:
import os

#os.environ["AWS_ACCESS_KEY_ID"]=<YOUR ACCESS KEY>
#os.environ["AWS_SECRET_ACCESS_KEY"]=<YOUR SECRET KEY>
#os.environ["AWS_SESSION_TOKEN"]=<OPTIONAL - YOUR SESSION TOKEN IF TEMP CREDENTIAL>
#os.environ["AWS_REGION"]=<AWS REGION WITH BEDROCK AGENTCORE AVAILABLE>

Strands Agents 및 Bedrock AgentCore Python SDK에 필요한 패키지를 설치합니다:

In [ ]:
#%pip install -q strands-agents strands-agents-tools bedrock-agentcore rich

AWS 리전에 따라 Nova Pro 모델 ID를 설정합니다:

In [ ]:
import boto3

region = boto3.session.Session().region_name

NOVA_PRO_MODEL_ID = "us.amazon.nova-pro-v1:0"
if region.startswith("eu"):
    NOVA_PRO_MODEL_ID = "eu.amazon.nova-pro-v1:0"
elif region.startswith("ap"):
    NOVA_PRO_MODEL_ID = "apac.amazon.nova-pro-v1:0"

print(f"Nova Pro Model ID: {NOVA_PRO_MODEL_ID}")

## Bedrock AgentCore Gateway란 무엇인가요?

Amazon Bedrock AgentCore Gateway는 OpenAPI 사양을 Model Context Protocol (MCP) 서버로 자동 변환하는 관리형 서비스입니다. 주요 이점은 다음과 같습니다:

- **자동 도구 생성**: 수동 코딩 없이 OpenAPI 엔드포인트를 MCP 도구로 변환
- **보안 인증**: 다양한 인증 방법(API 키, JWT, OAuth)에 대한 내장 지원
- **관리형 인프라**: 사용자 정의 MCP 서버를 배포하거나 유지 관리할 필요 없음
- **확장성**: 수요에 따라 자동으로 확장
- **통합**: 기존 REST API와의 원활한 통합

이 접근 방식을 통해 OpenAPI 문서가 있는 모든 REST API를 AI 에이전트의 도구로 빠르게 통합할 수 있습니다.

![openapi-gateway-apikey.png](images/openapi-gateway-apikey.png)

## OpenAPI를 대상으로 하는 AgentCore Gateway 생성

### 1단계: OpenAPI 사양 준비

MCP 도구를 자동으로 생성하는 데 사용될 Exa API OpenAPI 사양을 다운로드합니다. OpenAPI 사양은 사용 가능한 모든 엔드포인트, 매개변수 및 응답 스키마를 정의합니다.

In [ ]:
import requests
import os

url = "https://raw.githubusercontent.com/exa-labs/openapi-spec/refs/heads/master/exa-openapi-spec.yaml"
openapi_file_name = url.split("/")[-1]
save_path = f"./{openapi_file_name}"

if os.path.exists(save_path):
    print(f"파일이 이미 존재합니다: {save_path}")
else:
    response = requests.get(url)

    if response.status_code == 200:
        with open(save_path, 'wb') as file:
            file.write(response.content)
        print(f"파일이 성공적으로 다운로드되었습니다: {save_path}")
    else:
        print(f"파일 다운로드에 실패했습니다. 상태 코드: {response.status_code}")

### 2단계: OpenAPI 사양을 S3에 업로드

AgentCore Gateway가 자동 도구 생성을 위해 액세스할 수 있도록 OpenAPI 사양을 S3에 업로드합니다.

In [ ]:
import boto3

region = boto3.session.Session().region_name
# S3 클라이언트 생성
s3_client = boto3.client('s3', region_name=region)
sts_client = boto3.client('sts', region_name=region)

account_id = sts_client.get_caller_identity()["Account"]

# 매개변수 정의
# OpenAPI json 파일을 업로드할 S3 버킷
bucket_name = f'bedrock-agentcore-gateway-{account_id}-{region}'
file_path = f'./{openapi_file_name}'
object_key = openapi_file_name

# put_object를 사용하여 파일 업로드 및 응답 읽기
try:
    if region == "us-east-1":
        s3bucket = s3_client.create_bucket(
            Bucket=bucket_name
        )
    else:
        s3bucket = s3_client.create_bucket(
            Bucket=bucket_name,
            CreateBucketConfiguration={
                'LocationConstraint': region
            }
        )
    with open(file_path, 'rb') as file_data:
        response = s3_client.put_object(
            Bucket=bucket_name,
            Key=object_key,
            Body=file_data
        )

    # 계정 ID와 리전을 포함한 업로드된 객체의 ARN 구성
    openapi_s3_uri = f's3://{bucket_name}/{object_key}'
    print(f'업로드된 객체 S3 URI: {openapi_s3_uri}')
except Exception as e:
    print(f'파일 업로드 오류: {e}')
    with open(file_path, 'rb') as file_data:
        response = s3_client.put_object(
            Bucket=bucket_name,
            Key=object_key,
            Body=file_data
        )
    # 계정 ID와 리전을 포함한 업로드된 객체의 ARN 구성
    openapi_s3_uri = f's3://{bucket_name}/{object_key}'
    print(f'업로드된 객체 S3 URI: {openapi_s3_uri}')

### 3단계: Cognito 인바운드 인증을 사용하여 AgentCore Gateway 생성

먼저 AgentCore Gateway에 대한 보안 액세스를 위해 Cognito User Pool을 생성합니다. 

이 말은 client 가 agent 를 호출할 때 Cognito 인증을 통해 호출해야 한다는 것을 의미합니다.

이는 게이트웨이 엔드포인트에 대한 JWT 기반 인증을 제공합니다.

**생성되는 구성 요소:**
- **User Pool**: 사용자 ID 및 인증 관리
- **App Client**: 애플리케이션 수준 인증 활성화
- **Cognito Hosted Domain**: OAuth 2.0 토큰 엔드포인트를 위한 관리형 도메인 제공

그런 다음 Cognito JWT 인증을 사용하여 메인 AgentCore Gateway를 생성합니다. 이 게이트웨이는 OpenAPI 기반 대상을 호스팅합니다.

In [ ]:
from bedrock_agentcore_starter_toolkit.operations.gateway.client import GatewayClient
import boto3

region = boto3.session.Session().region_name

gateway_client = GatewayClient(region_name=region)

# Cognito OAuth 인증자 생성
cognito_response = gateway_client.create_oauth_authorizer_with_cognito("agentcore-gateway")

cognito_pool_id = cognito_response['client_info']['user_pool_id']
cognito_client_id = cognito_response['client_info']['client_id']
cognito_client_secret = cognito_response['client_info']['client_secret']
cognito_scope = cognito_response['client_info']['scope']
cognito_token_endpoint = cognito_response['client_info']['token_endpoint']
cognito_domain = cognito_response['client_info']['domain_prefix']
print(f"✅ User Pool ID: {cognito_pool_id}")
print(f"✅ Client ID: {cognito_client_id}")
print(f"✅ Client Secret: {cognito_client_secret}")
print(f"✅ Scope: {cognito_scope}")
print(f"✅ Token Endpoint: {cognito_token_endpoint}")
print(f"✅ Cognito Domain: {cognito_domain}")

# Cognito 인증자를 사용하여 게이트웨이 생성
gateway = gateway_client.create_mcp_gateway(authorizer_config=cognito_response["authorizer_config"])

gateway_id = gateway['gatewayId']
gateway_url = gateway['gatewayUrl']
print(f"✅ Gateway ID: {gateway_id}")
print(f"✅ Gateway URL: {gateway_url}")

### 4단계: OpenAPI 구성 및 아웃바운드 인증을 사용하여 게이트웨이 대상 생성

OpenAPI 사양을 사용하여 MCP 도구를 자동으로 생성하는 게이트웨이 대상을 생성하고, 

API 키는 아웃바운드 인증을 위해 Bedrock AgentCore Identity API Key Credential Provider에 안전하게 저장됩니다. 

대상 구성에는 다음이 포함됩니다:

- **OpenAPI 스키마**: S3에 저장된 OpenAPI 사양에 대한 참조
- **자격 증명 구성**: 외부 API와 인증하는 방법
- **매개변수 매핑**: API 키를 배치할 위치 (쿼리 매개변수, 헤더 등)

Exa API 키를 얻으려면 [Exa 로그인 페이지](https://dashboard.exa.ai/login)로 이동하여 이메일로 등록하세요.

그런 다음 Exa 대시보드의 [API 키 섹션](https://dashboard.exa.ai/api-keys)으로 이동하여 API 키를 생성하세요. API 키를 복사하여 아래 코드의 `EXA_API_KEY`에 입력하세요...

⚠️ **중요**: 플레이스홀더 API 키를 실제 Exa API 키로 교체하세요.

In [ ]:
gateway_target = gateway_client.create_mcp_gateway_target(
    gateway=gateway, 
    target_type="openApiSchema", 
    target_payload={
        "s3": {
            "uri": openapi_s3_uri
        }
    },
    credentials={
        # !-------- 여기에 EXA API 키를 업데이트하세요  --------!
        "api_key": "YOUR_EXA_API_KEY",
        "credential_location": "HEADER",
        "credential_parameter_name": "x-api-key"
    }
)
gateway_target_id = gateway_target['targetId']
credential_provider_name = gateway_target['credentialProviderConfigurations'][0]['credentialProvider']['apiKeyCredentialProvider']['providerArn'].split('/')[-1]

print(f"✅ Gateway Target ID: {gateway_id}")
print(f"✅ API Key Credential Provider Name: {credential_provider_name}")

## Strands Agent와 함께 배포된 AgentCore Gateway를 MCP 서버로 테스트

이제 적절한 인증을 사용하여 배포된 AgentCore Gateway를 MCP 서버로 테스트해보겠습니다.

### Cognito 인증에서 액세스 토큰 획득

AgentCore Gateway에 대한 요청을 인증하는 데 사용될 Cognito에서 JWT 액세스 토큰을 가져옵니다.

In [ ]:
# Cognito에서 액세스 토큰 가져오기
client_config = {
    "user_pool_id": cognito_pool_id,
    "client_id": cognito_client_id,
    "client_secret": cognito_client_secret,
    "scope": cognito_scope,
    "token_endpoint": cognito_token_endpoint,
    "region": region
}

token_response = gateway_client.get_access_token_for_cognito(client_config)
access_token = token_response
print(access_token)

### Strands Agent로 게이트웨이 테스트

이제 Strands Agent와 연결하여 AgentCore Gateway를 테스트해보겠습니다. 

게이트웨이는 OpenAPI 사양을 에이전트가 사용할 수 있는 MCP 도구로 자동 변환합니다.

**여기서 일어나는 일:**
1. JWT 베어러 토큰을 사용하여 게이트웨이에 연결
2. 사용 가능한 도구 목록 (OpenAPI 사양에서 자동 생성됨)
3. 이러한 도구에 액세스할 수 있는 에이전트 생성
4. 게이트웨이를 통해 외부 API를 사용하는 에이전트의 능력 테스트

In [ ]:
from strands import Agent
from strands.models import BedrockModel
from strands.tools.mcp import MCPClient
from mcp.client.streamable_http import streamablehttp_client

# Web Search MCP 서버에 연결
print("\nMCP 서버에 연결 중...")
headers = {
    "Authorization": f"Bearer {access_token}",
    #"Content-Type": "application/json"
}
exa_server = MCPClient(lambda: streamablehttp_client(gateway_url, headers))

with exa_server:
    mcp_tools = (exa_server.list_tools_sync())
    print(f"사용 가능한 도구: {[tool.tool_name for tool in mcp_tools]}")

    # 자체 구축 MCP 도구로 에이전트 생성
    agent = Agent(
        model=BedrockModel(model_id=NOVA_PRO_MODEL_ID),
        system_prompt = """당신은 간결한 응답을 제공하는 도움이 되는 어시스턴트입니다.""",
        tools=mcp_tools,
    )

    agent("Bedrock AgentCore란 무엇인가요?")

에이전트가 요청을 처리하고 응답을 생성하는 방법을 이해하기 위해 에이전트 루프의 상세한 실행 흐름을 살펴보겠습니다:

In [ ]:
from rich.table import Table
import rich
import json

console = rich.get_console()

console.print("에이전트 루프 세부 정보")
console.rule()
console.print(f"루프 수: {agent.event_loop_metrics.cycle_count}")

table = Table(title="에이전트 메시지", show_lines=True)
table.add_column("역할", style="green")
table.add_column("텍스트", style="magenta")
table.add_column("도구 이름", style="cyan")
table.add_column("도구 입력", style="cyan")
table.add_column("도구 결과", style="cyan")

for message in agent.messages:
    text = [content["text"] for content in message["content"] if "text" in content]
    tool_name = [content["toolUse"]["name"] for content in message["content"] if "toolUse" in content]
    tool_input = [content["toolUse"]["input"] for content in message["content"] if "toolUse" in content]
    tool_result = [content["toolResult"]["content"][0] for content in message["content"] if "toolResult" in content]
    table.add_row(message["role"], text[-1] if text else "", 
                  tool_name[-1] if tool_name else "", 
                  json.dumps(tool_input[-1], indent=2) if tool_input else "", 
                  (json.dumps(tool_result[-1], indent=2)[:500]+"\n.\n.\n." if len(str(tool_result[-1])) > 500 else json.dumps(tool_result[-1], indent=2)) if tool_result else "")

console.print(table)

## 리소스 정리 (선택사항)

배포된 리소스를 정리합니다:

In [ ]:
import boto3
import os

region = boto3.session.Session().region_name

agentcore_control_client = boto3.client('bedrock-agentcore-control', region_name=region)
cognito_client = boto3.client('cognito-idp', region_name=region)
iam_client = boto3.client('iam')
s3_client = boto3.client('s3', region_name=region)

try:
    print("AgentCore Gateway Target 삭제 중...")
    agentcore_control_client.delete_gateway_target(gatewayIdentifier=gateway_id, targetId=gateway_target_id)
    print("✓ AgentCore Gateway Target 삭제됨")
    
    print("AgentCore Gateway 삭제 중...")
    agentcore_control_client.delete_gateway(gatewayIdentifier=gateway_id)
    print("✓ AgentCore Gateway 삭제 시작됨")

    print("AgentCore Identity 삭제 중...")
    agentcore_control_client.delete_api_key_credential_provider(name=credential_provider_name)
    print("✓ AgentCore Identity 삭제 시작됨")

    print("Cognito User Pool 삭제 중...")
    cognito_client.delete_user_pool_domain(Domain=cognito_response['client_info']['domain_prefix'], UserPoolId=cognito_pool_id)
    cognito_client.delete_user_pool(UserPoolId=cognito_pool_id)
    print("✓ Cognito User Pool 삭제됨")

    print("S3 버킷 삭제 중...")
    s3_client.delete_object(Bucket=bucket_name, Key=openapi_file_name)
    s3_client.delete_bucket(Bucket=bucket_name)
    print("✓ S3 버킷 삭제됨")
except Exception as e:
    print(f"❌ 정리 중 오류 발생: {e}")
    print("일부 리소스를 수동으로 정리해야 할 수 있습니다.")

## 결론

이 실습에서 성공적으로 다음을 수행했습니다:

- ✅ JWT 인증을 사용하여 Bedrock AgentCore Gateway 생성
- ✅ 외부 API를 위한 보안 자격 증명 관리 구성
- ✅ OpenAPI 사양에서 MCP 도구 자동 생성
- ✅ AI 기반 API 상호작용을 위해 Strands Agents와 게이트웨이 통합

## AgentCore Gateway의 주요 이점

- **제로 코드 MCP 생성**: 모든 OpenAPI 사양을 MCP 도구로 자동 변환
- **보안 자격 증명 관리**: 다양한 인증 방법에 대한 내장 지원
- **관리형 인프라**: 사용자 정의 서버를 배포하거나 유지 관리할 필요 없음
- **확장 가능**: 로드 및 확장을 자동으로 처리
- **표준 기반**: OpenAPI 문서가 있는 모든 REST API와 작동